In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Code by MAMMAD ABBASLI    https://www.kaggle.com/mammadabbasli/friends-text-generator

In [ ]:
import warnings
import nltk
import spacy
import re
from spacy import displacy
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
from wordcloud import WordCloud
warnings.filterwarnings('ignore')

In [ ]:
path_of_file = '../input/feedback-prize-2021/train/00C819ADE423.txt'
text = open(path_of_file, 'r').read()

In [ ]:
text[:2000]

In [ ]:
nlp = spacy.load('en_core_web_sm')
stopword = nltk.corpus.stopwords.words('english')
def text_cleaning(text):
    
    text = re.sub(r'[^\w\s]', '',str(text))             #Punctuations
    text=re.split("\W+",text)                           #Tokenizing
    text=[word for word in text if word not in stopword]#Stop words
    text = ' '.join(text)                              
    return text


def frequent_of_words(string):
    
    clean_string = text_cleaning(string)
    split_string = pd.DataFrame(clean_string.split(),columns=['Words'])
    split_string = split_string.value_counts()[:1000].reset_index(drop=False)[:1000]
    split_string.columns = ['Words','Count']
    return split_string

In [ ]:
frequent_words = frequent_of_words(text)
frequent_words[:15].style.background_gradient(cmap='summer')

In [ ]:
fig = px.funnel(frequent_words[:15], x='Count', y='Words')
fig.show()

In [ ]:
list_for_cloud = []
for i in frequent_words.Words:
    list_for_cloud.append(str(i))

In [ ]:
wordcloud = WordCloud(width = 300, height = 300,
                background_color ='black',
                colormap='Set3',      
                stopwords = stopword,
                min_font_size = 10).generate(' '.join(list_for_cloud))
  
# plot the WordCloud image                       
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
name_list = ['Nasa','Venus','Earth','Solar','Mars','Planets']
scripts = []
split_string = text.split()
for name in name_list:
    scripts.append((name,split_string.count(name)))

In [ ]:
colors = ['#2F86A6','#34BE82','#2FDD92','#F2F013','#F9975D','#F4E185']
sections = [scripts[0][1],
            scripts[1][1],
            scripts[2][1],
            scripts[3][1],
           scripts[4][1],
           scripts[5][1]]
plt.figure(figsize=(14, 8), dpi=75)
plt.pie(sections, labels=name_list,colors=colors, 
        wedgeprops=dict( alpha=1),
        startangle=90,
        #explode = (0,0,0,0),
        autopct = '%0.1f%%',
         textprops={
                'fontsize': 15, 
                'fontweight': 'normal'}
            )

plt.axis('equal')
plt.title('Script Count',fontsize=20)
plt.show()

#After here only errors.

In [ ]:
def target(text):
    input_txt = text[:-1]
    target = text[1:]
    return input_txt, target

vocab = sorted(set(text))
char_index = {u:i for i, u in enumerate(vocab)}
index_of_charachter = np.array(vocab)
encoded_text = np.array([char_index[c] for c in text])
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequence_lenght = 120
#+1 because of zero indexing
sequences = char_dataset.batch(sequence_lenght+1, drop_remainder=True)
dataset = sequences.map(target)
batch_size = 128
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

#Function for Prepare Dataset for RNN Model

In [ ]:
class PlanetsTextGenerator:
    '''
    TextGeneratorModel writed for generate text for Planets.
    Class Bulit with Tenserflow.
    This class has multifunction. It can train and save alse test ability.
    '''

    def loss_func(self,true,pred):
         return sparse_categorical_crossentropy(true, pred, from_logits=True) 
        
    def base_model(self,size_of_vocab=94,embedding_dim = 64,neurons = 1026,batch_size = 128):
      
        model = Sequential()
        model.add(Embedding(size_of_vocab, embedding_dim,batch_input_shape=[batch_size, None]))
        model.add(GRU(neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
        model.add(Dense(size_of_vocab))
        model.compile(optimizer='adam', loss=self.loss_func) 
      
        return model
    
    def train_model(self,dataset,epoch=2,train_vb_size=94,em_dim = 64,rnn_nrs = 1026,batchs = 128):
        '''
        This funtion used for train model with defined params

        '''
    
        model = self.base_model(size_of_vocab = train_vb_size,
                                embedding_dim = em_dim,
                                neurons = rnn_nrs,
                                batch_size = batchs)
        print('\n    --Model Creat Succesfully--   ')
        model.fit(dataset,epochs=epoch)
        model.save('planets.h5') 
        print('\n    --Model Saved Succesfully--   ')
        return model
    
    def use_trained_model(self,path):
        '''
        Function takes only path of trained model and generated sentence afterwords

        '''
        saved_model = self.base_model(size_of_vocab=94,embedding_dim = 64,neurons = 1026,batch_size=1)
        saved_model.load_weights(path)
        saved_model.build(tf.TensorShape([1, None]))
        print('\n    --Model ReCreat Succesfully--   ')
        return saved_model
    
    def generate_dialog(self,model, start_sentence,characters=100):
        '''
        Purpose of this function is generate senctence
        model : trained sequential model
        start_sentence : according to which word will be generate
        characters : lenght of generated sentence
        '''
    
        num_generate = characters
        change_char = {'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6,'&': 7,"'": 8,'(': 9,')': 10, '*': 11,'+': 12,',': 13,'-': 14,'.': 15,'/': 16,'0': 17,'1': 18,'2': 19,'3': 20,'4': 21,'5': 22,'6': 23,'7': 24,
        '8': 25,'9': 26,':': 27,';': 28,'<': 29,'=': 30,'>': 31,'?': 32,'@': 33,'A': 34,'B': 35,'C': 36,'D': 37,'E': 38,'F': 39,'G': 40,'H': 41,'I': 42,'J': 43,'K': 44,'L': 45,'M': 46,'N': 47,'O': 48,'P': 49,
        'Q': 50,'R': 51,'S': 52,'T': 53,'U': 54,'V': 55,'W': 56,'X': 57,'Y': 58,'Z': 59,'[': 60,']': 61,'^': 62,'_': 63,'`': 64,'a': 65,'b': 66,'c': 67,'d': 68,'e': 69,'f': 70,'g': 71,'h': 72, 'i': 73,'j': 74,
        'k': 75,'l': 76,'m': 77,'n': 78,'o': 79,'p': 80,'q': 81,'r': 82,'s': 83,'t': 84,'u': 85,'v': 86,'w': 87,'x': 88,'y': 89,'z': 90,'{': 91,'|': 92, '}': 93}
        vocab = ['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
                 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}']
        
        index_of_charachter = np.array(vocab)
        # Vecotrizing sentecnce
        input_sentence = [change_char[chrr] for chrr in start_sentence]
        input_sentence = tf.expand_dims(input_sentence, 0)
        generated_txt = []
        model.reset_states()
        print('\n    --Dialog Creating--   ')
        for i in range(num_generate):
            
            predictions = model(input_sentence)
            predictions = tf.squeeze(predictions, 0)
            predictions = predictions / 1.0
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            input_sentence = tf.expand_dims([predicted_id], 0)
            generated_txt.append(index_of_charachter[predicted_id])
       
        return (start_sentence + ''.join(generated_txt))

In [ ]:
generator = PlanetsTextGenerator()

In [ ]:
first_model = generator.train_model(dataset,epoch=35)

In [ ]:
first_model.summary()

In [ ]:
path = './planets.h5'
test_model = generator.use_trained_model(path)